# Exploring Cospectral Graphs

Two graphs are **cospectral** if they have the same multiset of eigenvalues for a given matrix.

This notebook explores:
1. Constructing cospectral pairs using Godsil-McKay switching
2. Comparing different matrix types (adjacency, Laplacian, non-backtracking)
3. Understanding which matrices better distinguish graphs

This is a computational exploration - no API required.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

plt.style.use('seaborn-v0_8-whitegrid')
np.set_printoptions(precision=4, suppress=True)

## The Smallest Cospectral Pair

The smallest pair of non-isomorphic adjacency-cospectral graphs occurs at n=5.
These are $C_4 \cup K_1$ (4-cycle plus isolated vertex) and $K_{1,4}$ (star graph).

In [ ]:
# Construct the smallest cospectral pair
# C4 + K1 (4-cycle with isolated vertex)
G1 = nx.cycle_graph(4)
G1.add_node(4)  # isolated vertex

# K_{1,4} (star graph)
G2 = nx.star_graph(4)

print(f"G1 (C4 + K1): {G1.number_of_nodes()} vertices, {G1.number_of_edges()} edges")
print(f"G2 (K_{{1,4}}): {G2.number_of_nodes()} vertices, {G2.number_of_edges()} edges")

# Compute adjacency spectra
spec1 = np.sort(nx.adjacency_spectrum(G1).real)
spec2 = np.sort(nx.adjacency_spectrum(G2).real)

print("\nAdjacency eigenvalues:")
print(f"  G1: {spec1}")
print(f"  G2: {spec2}")
print(f"  Same spectrum: {np.allclose(spec1, spec2)}")

In [ ]:
# Visualize the pair
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

pos1 = nx.spring_layout(G1, seed=42)
pos2 = nx.spring_layout(G2, seed=42)

nx.draw(G1, pos1, ax=axes[0], with_labels=True, node_color='lightblue', 
        node_size=500, font_size=12, font_weight='bold')
axes[0].set_title(f'G1: C₄ + K₁\nSpectrum: {np.round(spec1, 2)}')

nx.draw(G2, pos2, ax=axes[1], with_labels=True, node_color='lightcoral',
        node_size=500, font_size=12, font_weight='bold')
axes[1].set_title(f'G2: K₁,₄ (Star)\nSpectrum: {np.round(spec2, 2)}')

plt.suptitle('Smallest Adjacency-Cospectral Pair (n=5)', fontsize=14)
plt.tight_layout()
plt.show()

## Can Other Matrices Distinguish Them?

Let's check if the Laplacian matrix can tell these graphs apart.

In [ ]:
# Laplacian spectra
lap1 = np.sort(nx.laplacian_spectrum(G1).real)
lap2 = np.sort(nx.laplacian_spectrum(G2).real)

print("Laplacian eigenvalues:")
print(f"  G1: {lap1}")
print(f"  G2: {lap2}")
print(f"  Same spectrum: {np.allclose(lap1, lap2)}")

if not np.allclose(lap1, lap2):
    print("\n✓ Laplacian DISTINGUISHES these graphs!")
else:
    print("\n✗ Laplacian does NOT distinguish these graphs")

## The Non-Backtracking Matrix

The **non-backtracking matrix** (Hashimoto matrix) is indexed by directed edges.
Entry $B_{e \to f} = 1$ if edge $e$ leads into edge $f$ without backtracking.

This matrix often distinguishes graphs that are cospectral for adjacency/Laplacian.

In [ ]:
def nonbacktracking_matrix(G: nx.Graph) -> np.ndarray:
    """Construct the non-backtracking (Hashimoto) matrix."""
    # Create directed edge list (both directions for undirected edges)
    directed_edges = []
    for u, v in G.edges():
        directed_edges.append((u, v))
        directed_edges.append((v, u))
    
    n_edges = len(directed_edges)
    edge_to_idx = {e: i for i, e in enumerate(directed_edges)}
    
    B = np.zeros((n_edges, n_edges))
    
    for i, (u, v) in enumerate(directed_edges):
        # Edge (u,v) can transition to edge (v,w) if w != u (no backtracking)
        for w in G.neighbors(v):
            if w != u:
                j = edge_to_idx[(v, w)]
                B[i, j] = 1
    
    return B

def nb_spectrum(G: nx.Graph) -> np.ndarray:
    """Compute non-backtracking eigenvalues."""
    if G.number_of_edges() == 0:
        return np.array([])
    B = nonbacktracking_matrix(G)
    eigs = np.linalg.eigvals(B)
    # Sort by magnitude, then by angle
    idx = np.lexsort((np.angle(eigs), np.abs(eigs)))
    return eigs[idx]

In [ ]:
# Compute NB spectra for our cospectral pair
nb1 = nb_spectrum(G1)
nb2 = nb_spectrum(G2)

print("Non-backtracking matrix sizes:")
print(f"  G1: {2*G1.number_of_edges()} x {2*G1.number_of_edges()}")
print(f"  G2: {2*G2.number_of_edges()} x {2*G2.number_of_edges()}")

print("\nNB eigenvalues (showing magnitudes):")
print(f"  G1: {np.sort(np.abs(nb1))[::-1]}")
print(f"  G2: {np.sort(np.abs(nb2))[::-1]}")

## Godsil-McKay Switching

Godsil-McKay switching is a graph operation that preserves the adjacency spectrum.
It can be used to construct cospectral pairs.

Given a partition of vertices into sets $C_1, ..., C_k, D$ where:
- Each $C_i$ induces a regular subgraph
- Each vertex in $D$ has 0, $|C_i|/2$, or $|C_i|$ neighbors in each $C_i$

We can switch edges between $D$ and the $C_i$ sets.

In [ ]:
def gm_switch(G: nx.Graph, C: set, D: set) -> nx.Graph:
    """
    Perform Godsil-McKay switching.
    
    For each vertex d in D that has exactly |C|/2 neighbors in C,
    switch its adjacencies: neighbors become non-neighbors and vice versa.
    """
    H = G.copy()
    half_C = len(C) // 2
    
    for d in D:
        neighbors_in_C = set(G.neighbors(d)) & C
        if len(neighbors_in_C) == half_C:
            # Switch: remove existing edges, add missing ones
            for c in C:
                if c in neighbors_in_C:
                    H.remove_edge(d, c)
                else:
                    H.add_edge(d, c)
    return H

# Example: construct a cospectral pair via GM-switching
# Start with the Petersen graph
P = nx.petersen_graph()

# One valid GM partition for Petersen graph
# C = {0, 1} forms a matching, D = {5, 6} 
C = {0, 1}
D = {5, 6}

# Check the partition is valid
for d in D:
    n_in_C = len(set(P.neighbors(d)) & C)
    print(f"Vertex {d} has {n_in_C} neighbors in C (need {len(C)//2})")

## Counting Cospectral Graphs by Matrix Type

Different matrices have different "resolving power". Let's empirically check
how many pairs of small graphs are cospectral for each matrix type.

In [ ]:
def spectra_match(s1: np.ndarray, s2: np.ndarray, tol: float = 1e-8) -> bool:
    """Check if two spectra are the same (up to tolerance)."""
    if len(s1) != len(s2):
        return False
    return np.allclose(np.sort(np.abs(s1)), np.sort(np.abs(s2)), atol=tol)

def analyze_cospectrality(graphs: list) -> dict:
    """Count cospectral pairs for different matrix types."""
    results = {'adj': 0, 'lap': 0, 'nb': 0}
    
    # Precompute spectra
    adj_spectra = [np.sort(nx.adjacency_spectrum(G).real) for G in graphs]
    lap_spectra = [np.sort(nx.laplacian_spectrum(G).real) for G in graphs]
    nb_spectra = [nb_spectrum(G) for G in graphs]
    
    # Count cospectral pairs
    for i, j in combinations(range(len(graphs)), 2):
        if not nx.is_isomorphic(graphs[i], graphs[j]):
            if spectra_match(adj_spectra[i], adj_spectra[j]):
                results['adj'] += 1
            if spectra_match(lap_spectra[i], lap_spectra[j]):
                results['lap'] += 1
            if spectra_match(nb_spectra[i], nb_spectra[j]):
                results['nb'] += 1
    
    return results

In [ ]:
# Generate all connected graphs on n=6 vertices
# (This uses networkx's graph_atlas which includes non-isomorphic graphs)
from networkx.generators.atlas import graph_atlas_g

# Get all graphs from atlas and filter by vertex count and connectivity
all_graphs = graph_atlas_g()
graphs_6 = [G for G in all_graphs 
            if G.number_of_nodes() == 6 and nx.is_connected(G)]

print(f"Number of connected graphs on 6 vertices: {len(graphs_6)}")

# Analyze cospectrality
results = analyze_cospectrality(graphs_6)

total_pairs = len(graphs_6) * (len(graphs_6) - 1) // 2
print(f"\nCospectral pairs out of {total_pairs} total non-isomorphic pairs:")
print(f"  Adjacency: {results['adj']}")
print(f"  Laplacian: {results['lap']}")
print(f"  Non-backtracking: {results['nb']}")

## Visualizing Spectral Distinguishing Power

Let's create a Venn-diagram style analysis showing which matrices distinguish which pairs.

In [ ]:
def detailed_cospectrality(graphs: list) -> dict:
    """Categorize cospectral pairs by which matrices distinguish them."""
    # Precompute spectra
    adj_spectra = [np.sort(nx.adjacency_spectrum(G).real) for G in graphs]
    lap_spectra = [np.sort(nx.laplacian_spectrum(G).real) for G in graphs]
    nb_spectra = [nb_spectrum(G) for G in graphs]
    
    categories = {
        'all_same': [],      # Cospectral for all three
        'adj_only': [],      # Same adj, different lap and nb
        'lap_only': [],      # Same lap, different adj and nb
        'adj_lap': [],       # Same adj and lap, different nb
        'adj_nb': [],        # Same adj and nb, different lap
        'lap_nb': [],        # Same lap and nb, different adj
    }
    
    for i, j in combinations(range(len(graphs)), 2):
        if not nx.is_isomorphic(graphs[i], graphs[j]):
            same_adj = spectra_match(adj_spectra[i], adj_spectra[j])
            same_lap = spectra_match(lap_spectra[i], lap_spectra[j])
            same_nb = spectra_match(nb_spectra[i], nb_spectra[j])
            
            if same_adj and same_lap and same_nb:
                categories['all_same'].append((i, j))
            elif same_adj and same_lap and not same_nb:
                categories['adj_lap'].append((i, j))
            elif same_adj and not same_lap and same_nb:
                categories['adj_nb'].append((i, j))
            elif not same_adj and same_lap and same_nb:
                categories['lap_nb'].append((i, j))
            elif same_adj and not same_lap and not same_nb:
                categories['adj_only'].append((i, j))
            elif not same_adj and same_lap and not same_nb:
                categories['lap_only'].append((i, j))
    
    return categories

cats = detailed_cospectrality(graphs_6)

print("Cospectral pair categories for n=6:")
for cat, pairs in cats.items():
    if pairs:
        print(f"  {cat}: {len(pairs)} pairs")

In [ ]:
# Visualize a pair that is adj+lap cospectral but NB-distinguished
if cats['adj_lap']:
    i, j = cats['adj_lap'][0]
    G1, G2 = graphs_6[i], graphs_6[j]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    pos1 = nx.spring_layout(G1, seed=42)
    pos2 = nx.spring_layout(G2, seed=42)
    
    nx.draw(G1, pos1, ax=axes[0], with_labels=True, node_color='lightblue',
            node_size=500, font_weight='bold')
    axes[0].set_title(f'Graph {i}\n{G1.number_of_edges()} edges')
    
    nx.draw(G2, pos2, ax=axes[1], with_labels=True, node_color='lightcoral',
            node_size=500, font_weight='bold')
    axes[1].set_title(f'Graph {j}\n{G2.number_of_edges()} edges')
    
    plt.suptitle('Adj+Lap Cospectral, NB-Distinguished', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"\nAdjacency spectrum: {np.round(np.sort(nx.adjacency_spectrum(G1).real), 4)}")
    print("(Same for both graphs)")
    print("\nNB spectral radii:")
    print(f"  G1: {np.max(np.abs(nb_spectrum(G1))):.4f}")
    print(f"  G2: {np.max(np.abs(nb_spectrum(G2))):.4f}")

## Summary

Key findings from this exploration:

1. **Cospectral graphs exist**: Non-isomorphic graphs can share the same spectrum
2. **Matrix choice matters**: Different matrices have different resolving power
3. **Non-backtracking advantage**: The NB matrix often distinguishes adj-cospectral pairs
4. **GM-switching**: A systematic way to construct cospectral pairs

The SMOL database contains precomputed spectra for all four matrix types,
making it easy to find and study cospectral families at scale.